In [2]:
import numpy as np
import gudhi
import mdtraj as md
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import *
from math import pi
import MDAnalysis as mda
from MDAnalysis.analysis.base import (AnalysisBase,
                                      AnalysisFromFunction,
                                      analysis_class)
import pandas as pd
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.cluster import spectral_clustering
import random
import gudhi.representations

/home/crystal/miniconda3/envs/openmm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
traj6 = md.load('ahelix100frame.pdb')
traj6 = md.Trajectory.superpose(traj6, traj6[0], frame=0, atom_indices=None, ref_atom_indices=None, parallel=True)

In [5]:
carbon_atoms = traj6.topology.select("element == 'C'")

In [6]:
num_frames = traj6.n_frames

In [ ]:
distance_matrix = np.zeros((num_frames, num_frames))

for frame1 in range(num_frames):
    persistence_pairs_frame1 = []
    carbon_coordinates_frame1 = traj6.xyz[frame1, carbon_atoms, :]
    rips_complex_frame1 = gudhi.RipsComplex(points=carbon_coordinates_frame1)
    simplex_tree_frame1 = rips_complex_frame1.create_simplex_tree(max_dimension=3)
    persistence_frame1 = simplex_tree_frame1.persistence()
    persistence_pairs1 = simplex_tree_frame1.persistence_intervals_in_dimension(1)
    persistence_diagrams1 = []
    persistence_diagrams1.append(persistence_pairs1)
    print(f'{frame1}')
    for frame2 in range(frame1 + 1, num_frames):
        persistence_pairs_frame2 = []
        carbon_coordinates_frame2 = traj6.xyz[frame2, carbon_atoms, :]
        rips_complex_frame2 = gudhi.RipsComplex(points=carbon_coordinates_frame2)
        simplex_tree_frame2 = rips_complex_frame2.create_simplex_tree(max_dimension=3)
        persistence_frame2 = simplex_tree_frame2.persistence()
        persistence_pairs2 = simplex_tree_frame2.persistence_intervals_in_dimension(1)
        persistence_diagrams2 = []
        persistence_diagrams2.append(persistence_pairs2)
        bottleneck_distance = gudhi.bottleneck_distance(persistence_pairs1, persistence_pairs2)

        distance_matrix[frame1, frame2] = bottleneck_distance

In [ ]:
np.save('ahelix100H1.npy',distance_matrix)